In [19]:
import pointblank as pb
import pandas as pd
import os

In [20]:
df = pd.read_csv("../data/drugs.csv")

In [21]:
print(list(df.columns))
print(len(df.columns))

['STATE', 'STATENAME', 'ST_CASE', 'VEH_NO', 'PER_NO', 'DRUGSPEC', 'DRUGSPECNAME', 'DRUGMETHOD', 'DRUGMETHODNAME', 'DRUGRES', 'DRUGRESNAME', 'DRUGQTY', 'DRUGQTYNAME', 'DRUGACTQTY', 'DRUGACTQTYNAME', 'DRUGUOM', 'DRUGUOMNAME']
17


In [22]:
expected_schema = pb.Schema(
    columns=[
    ('STATE' , 'int64'),
    ('STATENAME', 'object'),
    ('ST_CASE', 'int64'),
    ('VEH_NO', 'int64'),
    ('PER_NO', 'int64'),
    ('DRUGSPEC', 'int64'),
    ('DRUGSPECNAME', 'object'),
    ('DRUGMETHOD', 'int64'),
    ('DRUGMETHODNAME', 'object'),
    ('DRUGRES', 'int64'),
    ('DRUGRESNAME', 'object'),
    ('DRUGQTY', 'int64'),
    ('DRUGQTYNAME', 'object'),
    ('DRUGACTQTY', 'float64'),
    ('DRUGACTQTYNAME', 'object'),
    ('DRUGUOM', 'int64'),
    ('DRUGUOMNAME', 'object'),
    ('DRUGMETHOD_valid', 'int64')
    ])

# The DRUGMETHOD field is valid if its value is either:
# - In the range 1 to 26 (inclusive), or
# - One of the specific codes: 0, 96, 97, or 99
#
# Applying these two validation rules separately using:
#   .col_vals_between(columns="DRUGMETHOD", left=1, right=26)
#   .col_vals_in_set(columns="DRUGMETHOD", set=[00,96,97,99])
# results in partial failures, since each rule only validates part of the allowed values.
#
# To address this, we combined both conditions into a single boolean column (`DRUGMETHOD_valid`)
# indicating whether each row meets either condition.
#
# Since pointblank does not currently support validation on boolean columns,
# we convert the boolean values to integers (True → 1, False → 0),
# and then validate that all values in the new column equal 1.

df['DRUGMETHOD_valid'] = (df['DRUGMETHOD'].isin([00,96,97,99]) | ((df['DRUGMETHOD'] >=1) & (df['DRUGMETHOD'] <= 26))).astype(int)

validation_report = (
    pb.Validate(
        df, 
        label="Validate Drugs Data Report",
        thresholds=pb.Thresholds(warning=1, error= 20, critical=0.10)
        )
    .col_vals_between(columns="STATE", left=1, right=56)    # Step 1
    .col_vals_regex(columns="STATENAME", pattern=r'^[A-Za-z ]+$')   # Step 2
    .col_vals_between(columns='VEH_NO', left=0, right=99)   # Step 3
    .col_vals_between(columns='PER_NO', left=1, right=99)   # Step 4
    .col_vals_in_set(columns="DRUGSPEC", set=[0,1,2,11,12,13,14,15,96,97,98,99])    # Step 5
    .col_vals_eq(columns='DRUGMETHOD_valid', value=1)   # Step 6
    .col_vals_in_set(columns="DRUGQTY", set=[0,1,2,3,4,96,97,98,99])    # Step 7
    .col_vals_in_set(columns='DRUGUOM', set=[1,2,3,4,5,6,7,8,-9])   # Step 8
    .col_vals_not_null(columns=["PER_NO","VEH_NO","STATENAME"]) # Step 9, 10, 11 for 3 columns
    # 17 given column plus 1 additional column
    .col_count_match(18)    # Step 12
    .col_schema_match(schema = expected_schema) # Step 13
    .interrogate()
)

validation_report

Validate(data=        STATE STATENAME  ST_CASE  VEH_NO  PER_NO  DRUGSPEC    DRUGSPECNAME  \
0           1   Alabama    10001       1       1         0  Test Not Given   
1           1   Alabama    10002       1       1         1     Whole Blood   
2           1   Alabama    10002       1       1         1     Whole Blood   
3           1   Alabama    10002       1       1         1     Whole Blood   
4           1   Alabama    10002       1       1         1     Whole Blood   
...       ...       ...      ...     ...     ...       ...             ...   
129624     56   Wyoming   560120       1       4         0  Test Not Given   
129625     56   Wyoming   560120       1       5         0  Test Not Given   
129626     56   Wyoming   560120       2       1         0  Test Not Given   
129627     56   Wyoming   560121       1       1         1     Whole Blood   
129628     56   Wyoming   560121       1       1         1     Whole Blood   

        DRUGMETHOD                    DRUGMETHODNAME  DRUGRES  \
0                0                    Test Not Given        0   
1               19  Unknown Confirmatory Test Method     3033   
2               19  Unknown Confirmatory Test Method     3022   
3               19  Unknown Confirmatory Test Method     3053   
4               19  Unknown Confirmatory Test Method     5060   
...            ...                               ...      ...   
129624           0                    Test Not Given        0   
129625           0                    Test Not Given        0   
129626           0                    Test Not Given        0   
129627           9     Unknown Screening Test Method     3026   
129628           9     Unknown Screening Test Method     3034   

                               DRUGRESNAME  DRUGQTY           DRUGQTYNAME  \
0                           Test Not Given        0        Test Not Given   
1                                  Cocaine        2  Actual Drug Quantity   
2                          Benzoylecgonine        2  Actual Drug Quantity   
3                          Methamphetamine        2  Actual Drug Quantity   
4       Delta 9-tetrahydrocannabinol [THC]        2  Actual Drug Quantity   
...                                    ...      ...                   ...   
129624                      Test Not Given        0        Test Not Given   
129625                      Test Not Given        0        Test Not Given   
129626                      Test Not Given        0        Test Not Given   
129627                            Caffeine        3  Presumptive Positive   
129628                            Cotinine        3  Presumptive Positive   

        DRUGACTQTY  DRUGACTQTYNAME  DRUGUOM     DRUGUOMNAME  DRUGMETHOD_valid  
0            -99.0  Not Applicable       -9  Not Applicable                 1  
1             20.0          20.000        5           ng/mL                 1  
2             39.0          39.000        5           ng/mL                 1  
3            200.0         200.000        5           ng/mL                 1  
4              7.9           7.900        5           ng/mL                 1  
...            ...             ...      ...             ...               ...  
129624       -99.0  Not Applicable       -9  Not Applicable                 1  
129625       -99.0  Not Applicable       -9  Not Applicable                 1  
129626       -99.0  Not Applicable       -9  Not Applicable                 1  
129627       -99.0  Not Applicable       -9  Not Applicable                 1  
129628       -99.0  Not Applicable       -9  Not Applicable                 1  

[129629 rows x 18 columns], tbl_name=None, label='Validate Drugs Data Report', thresholds=Thresholds(warning=1, error=20, critical=0.1), actions=None, final_actions=None, brief=None, lang='en', locale='en')

In [23]:

# Export Validation report to html
result_folder = "../validation_result"
os.makedirs(result_folder, exist_ok=True)
validation_report.get_tabular_report().write_raw_html(f"{result_folder}/drugs_data_validation_report.html")


In [24]:
# i refers to the index (position) of the validation step in the pointblank.Validate() pipeline starting from 1
validation_report.get_step_report(i=3)

GT(_tbl_data=        _row_num_  STATE STATENAME  ST_CASE  VEH_NO  PER_NO  DRUGSPEC  \
0               1      1   Alabama    10001       1       1         0   
1               2      1   Alabama    10002       1       1         1   
2               3      1   Alabama    10002       1       1         1   
3               4      1   Alabama    10002       1       1         1   
4               5      1   Alabama    10002       1       1         1   
129624     129625     56   Wyoming   560120       1       4         0   
129625     129626     56   Wyoming   560120       1       5         0   
129626     129627     56   Wyoming   560120       2       1         0   
129627     129628     56   Wyoming   560121       1       1         1   
129628     129629     56   Wyoming   560121       1       1         1   

          DRUGSPECNAME  DRUGMETHOD                    DRUGMETHODNAME  DRUGRES  \
0       Test Not Given           0                    Test Not Given        0   
1          Whole Blood          19  Unknown Confirmatory Test Method     3033   
2          Whole Blood          19  Unknown Confirmatory Test Method     3022   
3          Whole Blood          19  Unknown Confirmatory Test Method     3053   
4          Whole Blood          19  Unknown Confirmatory Test Method     5060   
129624  Test Not Given           0                    Test Not Given        0   
129625  Test Not Given           0                    Test Not Given        0   
129626  Test Not Given           0                    Test Not Given        0   
129627     Whole Blood           9     Unknown Screening Test Method     3026   
129628     Whole Blood           9     Unknown Screening Test Method     3034   

                               DRUGRESNAME  DRUGQTY           DRUGQTYNAME  \
0                           Test Not Given        0        Test Not Given   
1                                  Cocaine        2  Actual Drug Quantity   
2                          Benzoylecgonine        2  Actual Drug Quantity   
3                          Methamphetamine        2  Actual Drug Quantity   
4       Delta 9-tetrahydrocannabinol [THC]        2  Actual Drug Quantity   
129624                      Test Not Given        0        Test Not Given   
129625                      Test Not Given        0        Test Not Given   
129626                      Test Not Given        0        Test Not Given   
129627                            Caffeine        3  Presumptive Positive   
129628                            Cotinine        3  Presumptive Positive   

        DRUGACTQTY  DRUGACTQTYNAME  DRUGUOM     DRUGUOMNAME  DRUGMETHOD_valid  
0            -99.0  Not Applicable       -9  Not Applicable                 1  
1             20.0          20.000        5           ng/mL                 1  
2             39.0          39.000        5           ng/mL                 1  
3            200.0         200.000        5           ng/mL                 1  
4              7.9           7.900        5           ng/mL                 1  
129624       -99.0  Not Applicable       -9  Not Applicable                 1  
129625       -99.0  Not Applicable       -9  Not Applicable                 1  
129626       -99.0  Not Applicable       -9  Not Applicable                 1  
129627       -99.0  Not Applicable       -9  Not Applicable                 1  
129628       -99.0  Not Applicable       -9  Not Applicable                 1  , _body=<great_tables._gt_data.Body object at 0x000001BBBAEDE900>, _boxhead=Boxhead([ColInfo(var='_row_num_', type=<ColInfoTypeEnum.default: 1>, column_label='', column_align='right', column_width='56.8px'), ColInfo(var='STATE', type=<ColInfoTypeEnum.default: 1>, column_label=Html(text="<div><div style='white-space: nowrap; text-overflow: ellipsis; overflow: hidden; padding-bottom: 2px; margin-bottom: 2px;'>STATE</div><div style='white-space: nowrap; text-overflow: ellipsis; overflow: hidden; padding-top: 2px; margin-top: 2px;'><em>int64</em></div></div>"), 